In [0]:
#Connection to ADLS bus01prod02 container where pilot data is present
storage_account = "adlsweudpbus01prod02"
key=dbutils.secrets.get(scope="credentials", key="SP-Password")
id=dbutils.secrets.get(scope="credentials", key="clientId")
spark.conf.set("fs.azure.account.auth.type." + storage_account + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storage_account + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storage_account + ".dfs.core.windows.net", id)
spark.conf.set("fs.azure.account.oauth2.client.secret." + storage_account + ".dfs.core.windows.net", key)
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storage_account + ".dfs.core.windows.net", "https://login.microsoftonline.com/505cca53-5750-4134-9501-8d52d5df3cd1/oauth2/token")

%run /Workspace/Repos/kroeper@tbdir.net/Utils/Storage_connection

import sys
sys.path.insert(0,"/Workspace/Repos/Misc/Utils/")

from storageConnection import *
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql import types
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import when



conn = StorageConnection('adls_ctp', 'sql_rd')
path = "abfss://al-ctp-pilot-contracts@adlsweudpbus01prod02.dfs.core.windows.net/data/"
df = spark.read.format("delta").load(path)



In [0]:
device_ids = [
    'ctp-0020008236', 'ctp-0020008009', 'ctp-0020008214', 'ctp-0020006799',
    'ctp-0020007916', 'ctp-0020006875', 'ctp-0020007025', 'ctp-0020007843',
    'ctp-0020007885', 'ctp-0020007877', 'ctp-0020008119', 'ctp-0440008057'
]
start_date = '2022-12-01'
end_date = '2023-11-30'
temp_id = '4378'        #Air Temperature Outside Current value SCA
standstill_id = '23'
plug_sig = '239'
prec_sig = '243'
temp_inside = '219'     #Air Temperature Inside Current value HVAC
#ChargingStateDC = '321'
#ChargingStateAC = '320'
#Status_Plugin_Charging = '239'

In [0]:
import time
from datetime import datetime, timedelta

df.filter(F.col('SignalID') == Status_Plugin_Charging).filter((df['SignalDate'] >= '2022-12-22') & (df['SignalDate'] <= '2023-11-30')).filter(df['DeviceID'].isin(device_ids)).select('DeviceID', 'SignalTimestamp').groupBy('DeviceID').agg(F.max('SignalTimestamp'), F.min('SignalTimestamp')).display()

In [0]:
import time
from datetime import datetime, timedelta

df.filter((F.col('SignalID') == '243')).filter((df['SignalDate'] >= '2022-11-22') & (df['SignalDate'] <= '2023-11-30')).filter(df['DeviceID'].isin(device_ids)).groupBy('DeviceID').agg(F.max('SignalTimestamp'), F.min('SignalTimestamp')).display()

In [0]:
df.filter((F.col('SignalID') == '243') | (F.col('SignalID') == '239')).filter((df['SignalDate'] >= '2023-11-22') & (df['SignalDate'] <= '2023-11-30')).orderBy('SignalTimestamp').select('DeviceID').distinct().display()

In [0]:
df = df.filter((F.col('SignalID') == temp_id) | (F.col('SignalID') == standstill_id) | (F.col('SignalID') == plug_sig) | (F.col('SignalID') == prec_sig) | (F.col('SignalID') == temp_inside)).filter(df['SignalValue'] <= 50.0).filter(df['SignalValue'] >= -20.0).filter((df['SignalDate'] >= start_date) & (df['SignalDate'] <= end_date)).filter(df['DeviceID'].isin(device_ids)).orderBy('SignalTimestamp').select('DeviceID', 'CustomerName', 'SignalTimestamp','SignalID', 'SignalValue')
#df.display()


In [0]:
df = df.groupBy('DeviceID', 'CustomerName', 'SignalTimestamp').pivot('SignalID', ['23', '4378', '239', '243', '219']).agg(F.first('SignalValue'))
#df.display()


In [0]:
window = Window.partitionBy("DeviceID").orderBy("SignalTimestamp").rowsBetween(-sys.maxsize, 0)
window2 = Window.partitionBy("DeviceID").orderBy('SignalTimestamp')
window3 = Window.partitionBy("DeviceID").orderBy("SignalTimestamp").rowsBetween( 0,120)


df = df.withColumn('4378', F.last('4378', ignorenulls = True).over(window))
#df = df.withColumn('4378', F.when(F.col('4378') == None, F.first('4378', ignorenulls = True).over(window3)))
#df.display()

#forward filling
df = df.withColumn('4378', F.col('4378').cast('float'))
df = df.withColumn('239', F.last('239', ignorenulls = True).over(window))
df = df.withColumn('243', F.last('243', ignorenulls = True).over(window))
df = df.withColumn('219', F.last('219', ignorenulls = True).over(window))
df = df.withColumn('23', F.last('23', ignorenulls = True).over(window))

#df = df.withColumn('23', F.when(F.col('23') == None, F.first('23', ignorenulls = True).over(window3)))

df = df.withColumn('lag_23', F.lag('23').over(window2))

#creating ID
df = df.withColumn('ID', F.when(((F.col('lag_23') == 0) | (F.col('lag_23') == None)) & (F.col('23') == 1), F.sum('23').over(window)))
#df.display()

#forward filling ID
df = df.withColumn('ID', F.last('ID', ignorenulls = True).over(window))
#df.display()

#filtering all events in which the vehicle is not standing still
df = df.filter(~((F.col('lag_23') == 0) & (F.col('23') == 0)))

#df.display()

#getting if preconditioning was done within a particular ID. If prec was done and not done within an ID then done counts
df = df.groupBy('DeviceID','CustomerName', 'ID').agg(F.avg('4378').alias('Avg_Outside_Temp'), ((unix_timestamp(F.max('SignalTimestamp')) - unix_timestamp(F.min('SignalTimestamp'))) / 60).alias('Time_Difference'), F.max('239').alias('Plug'), F.max(F.when(F.col('243')== 1, 4).otherwise(F.col('243'))).alias('Preconditioning'), F.avg('219').alias('Avg_Inside_Temp'))#.orderBy('DeviceID', 'ID')
df = df.filter(df['ID'].isNotNull()).filter(F.col('Time_Difference') >= 1)
winid = Window.partitionBy('DeviceID').orderBy('ID').rowsBetween(0,100)
df = df.withColumn('Avg_Outside_Temp',  F.first('Avg_Outside_Temp', ignorenulls = True).over(winid))
#df.display()


In [0]:
#rounding the values for the temperature and time difference
df = df.filter(F.col('Plug') == 1).withColumn('Avg_Outside_Temp', (df['Avg_Outside_Temp'] / 5).cast('int')).withColumn('Avg_Inside_Temp', (df['Avg_Inside_Temp'] / 5).cast('int')).withColumn('Time_Difference', (df['Time_Difference'] / 60).cast('int'))
df = df.withColumn('Time_Difference', when(F.col('Time_Difference') >= 5, '>= 5').otherwise(F.col('Time_Difference')))
df = df.withColumn('Avg_Outside_Temp', df['Avg_Outside_Temp'] * 5).withColumn('Avg_Inside_Temp', df['Avg_Inside_Temp'] * 5)
df = df.withColumn('Time_Difference', F.when(F.col('Time_Difference') == None, 0).otherwise(F.col('Time_Difference')))
#df.display()

In [0]:
df = df.filter(~(F.col('Preconditioning') == 'null'))
df.display()

In [0]:
conn.sql_rd.write_table(data=df,table_name='rd.CTP_Sales_Standstill_plug_prec_inTemp',mode='append',truncate=False)
conn.sql_rd.execute_query()

In [0]:
df = df.groupBy('Avg_Outside_Temp', 'DeviceID', 'CustomerName').pivot('Time_Difference').count()
df = df.orderBy('Avg_Outside_Temp')
df.display()

In [0]:
df = df.withColumn('Avg_Outside_Temp', df['Avg_Outside_Temp'] * 5)
df.groupBy('Avg_Outside_Temp').sum().orderBy('Avg_Outside_Temp').display()

In [0]:
df.display()

In [0]:
conn.sql_rd.write_table(data=df,table_name='rd.CTP_Sales_Standstill_plug_detected',mode='append',truncate=False)

In [0]:
df = conn.sql_rd.read_table(table_name='rd.CTP_Sales_Standstill')
df.display()

In [0]:
df.groupBy('Avg_Temperature').sum().orderBy('Avg_Temperature').display()

In [0]:
df.filter(F.col('DeviceID') == 'ctp-0440008814').display()
df.select('CustomerName').distinct().display()

Avg_Temperature,DeviceID,CustomerName,0,1,2,3,4,>= 5
-1,ctp-0440008814,üstra Hannoversche Verkehrsbetriebe AG,2,null,null,1,null,null
0,ctp-0440008814,üstra Hannoversche Verkehrsbetriebe AG,273,16,4,1,2,29
1,ctp-0440008814,üstra Hannoversche Verkehrsbetriebe AG,501,18,2,4,2,35
2,ctp-0440008814,üstra Hannoversche Verkehrsbetriebe AG,451,8,4,5,8,31
3,ctp-0440008814,üstra Hannoversche Verkehrsbetriebe AG,127,4,3,null,1,12
4,ctp-0440008814,üstra Hannoversche Verkehrsbetriebe AG,18,1,null,null,null,3


CustomerName
Hamburger Hochbahn AG
Stadtverkehr Lübeck GmbH
ESWE Verkehrsgesellschaft mbH
üstra Hannoversche Verkehrsbetriebe AG
